In [45]:
import os 
import cv2 
import numpy as np 
import requests 
import torch 
import torch.onnx 
from torch import nn 

torch.asinh is already binded with onnx op asinh

In [46]:
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
    
    def forward(self, x):
        return torch.asinh(x)

In [47]:
model = MyModel()
input1 = torch.randn(1, 1, 3, 3)
torch.onnx.export(model, input1, "asinh.onnx", verbose=True)

Exported graph: graph(%onnx::Asinh_0 : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu)):
  %1 : Float(1, 1, 3, 3, strides=[9, 9, 3, 1], requires_grad=0, device=cpu) = onnx::Asinh[onnx_name="/Asinh"](%onnx::Asinh_0), scope: __main__.MyModel:: # /tmp/ipykernel_1991/203262611.py:6:0
  return (%1)



### 自定义ONNX算子

torchvision.ops.DeformConv2d

```py
torch.ops.torchvision.deform_conv2d(
        input,
        weight,
        offset,
        mask,
        bias,
        stride_h,
        stride_w,
        pad_h,
        pad_w,
        dil_h,
        dil_w,
        n_weight_grps,
        n_offset_grps,
        use_mask,
    )
```

In [48]:
import torchvision as tv

class DefConv2d(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = torch.nn.Conv2d(3, 18, 3)
        self.conv2 = tv.ops.DeformConv2d(3, 3, 3)
    
    def forward(self, x):
        return self.conv2(x, self.conv1(x))

In [49]:
from torch.onnx.symbolic_helper import parse_args

@parse_args('v', 'v', 'v', 'v', 'v', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'i', 'none')
def symbolic(g,
             input, weight, offset, mask, bias,
             stride_h, stride_w,
             pad_h, pad_w,
             dil_h, dil_w,
             n_weight_grps, n_offset_grps,
             use_mask):
    return g.op("custom::deform_conv2d", input, offset)

In [50]:
from torch.onnx import register_custom_op_symbolic

register_custom_op_symbolic("torchvision::deform_conv2d", symbolic, 9)


In [51]:
model = DefConv2d()
input = torch.randn(1, 3, 10, 10)
torch.onnx.export(model, input, "deform.onnx", verbose=True)

Exported graph: graph(%input : Float(1, 3, 10, 10, strides=[300, 100, 10, 1], requires_grad=0, device=cpu),
      %conv1.weight : Float(18, 3, 3, 3, strides=[27, 9, 3, 1], requires_grad=1, device=cpu),
      %conv1.bias : Float(18, strides=[1], requires_grad=1, device=cpu)):
  %/conv1/Conv_output_0 : Float(1, 18, 8, 8, strides=[1152, 64, 8, 1], requires_grad=0, device=cpu) = onnx::Conv[dilations=[1, 1], group=1, kernel_shape=[3, 3], pads=[0, 0, 0, 0], strides=[1, 1], onnx_name="/conv1/Conv"](%input, %conv1.weight, %conv1.bias), scope: __main__.DefConv2d::/torch.nn.modules.conv.Conv2d::conv1 # /root/projects/modelDep/venv/lib/python3.10/site-packages/torch/nn/modules/conv.py:456:0
  %6 : Float(*, *, *, *, strides=[192, 64, 8, 1], requires_grad=1, device=cpu) = custom::deform_conv2d[onnx_name="/conv2/deform_conv2d"](%input, %/conv1/Conv_output_0), scope: __main__.DefConv2d::/torchvision.ops.deform_conv.DeformConv2d::conv2 # /root/projects/modelDep/venv/lib/python3.10/site-packages/torch/

[W shape_type_inference.cpp:1968] Warning: The shape inference of custom::deform_conv2d type is missing, so it may result in wrong shape inference for the exported graph. Please consider adding it in symbolic function. (function UpdateReliable)


### 使用`torch.autograd.Function`

In [53]:
from typing import Any
import my_lib

class MyAdd(torch.autograd.Function):

    @staticmethod
    def forward(ctx, a, b):
        return my_lib.my_add(a, b)
    
    @staticmethod
    def symbolic(g, a, b):
        two = g.op("Constant", value_t=torch.tensor(2))
        a = g.op("Mul", a, two)
        return g.op("Add", a, b)

In [54]:
my_add = MyAdd.apply

class MyAddModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, a, b):
        return my_add(a, b)

In [57]:
model = MyAddModule()
input = torch.randn(1, 3, 10, 10)
torch.onnx.export(model, (input, input), "my_add.onnx", verbose=True,
                  input_names=["a", "b"], output_names=["output"])
torch_output = model(input, input).detach().numpy()

import onnxruntime
sess = onnxruntime.InferenceSession("my_add.onnx")
ort_output = sess.run(None, {"a": input.numpy(), "b": input.numpy()})

Exported graph: graph(%a : Float(1, 3, 10, 10, strides=[300, 100, 10, 1], requires_grad=0, device=cpu),
      %b : Float(1, 3, 10, 10, strides=[300, 100, 10, 1], requires_grad=0, device=cpu)):
  %/Constant_output_0 : Float(requires_grad=0, device=cpu) = onnx::Constant[value={2}, onnx_name="/Constant"](), scope: __main__.MyAddModule::
  %/Mul_output_0 : Float(1, 3, 10, 10, strides=[300, 100, 10, 1], device=cpu) = onnx::Mul[onnx_name="/Mul"](%a, %/Constant_output_0), scope: __main__.MyAddModule::
  %output : Float(1, 3, 10, 10, strides=[300, 100, 10, 1], requires_grad=0, device=cpu) = onnx::Add[onnx_name="/Add"](%/Mul_output_0, %b), scope: __main__.MyAddModule:: # /root/projects/modelDep/venv/lib/python3.10/site-packages/torch/autograd/function.py:598:0
  return (%output)



In [61]:
print(np.array(ort_output).shape)

(1, 1, 3, 10, 10)
